# Wasserstein GAN

The Wasserstein GAN [(paper here)](https://arxiv.org/abs/1701.07875) is a improve GAN model that improves learning 
stability and helps get rid of problems like mode collapse, which we experienced with our first GAN model.

In [1]:
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.autograd as autograd
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

manualSeed = random.randint(1, 10000)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

print("Seed:", manualSeed)

In [2]:
batch_size = 64
img_shape = (3, 64, 64)

# Create the Datasets

transform = transforms.Compose(
    [
        transforms.Resize(64),
        transforms.CenterCrop(64),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

dataset_mountains = dset.ImageFolder(root="../data/mountains", transform=transform)
dataset_two = dset.ImageFolder(root="../data/mountains", transform=transform)
dataset_three = dset.ImageFolder(root="../data/mountains", transform=transform)

dataloader_mountains = torch.utils.data.DataLoader(dataset_mountains, batch_size=batch_size, shuffle=True, num_workers=2)
dataloader_two = torch.utils.data.DataLoader(dataset_two, batch_size=batch_size, shuffle=True, num_workers=2)
dataloader_three = torch.utils.data.DataLoader(dataset_three, batch_size=batch_size, shuffle=True, num_workers=2)

# Set To Zero For CPU Mode

ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu == 1) else "cpu")

print("Running On Device:", device)

Will run on device cuda:0


In [3]:
class Generator(nn.Module):
    
    def __init__(self, ngpu):
        
        super(Generator, self).__init__()

        self.ngpu = ngpu
        
        def block(in_feat, out_feat, normalize=True):
            
            layers = [nn.Linear(in_feat, out_feat)]
            
            if normalize:
                
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(100, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img

In [4]:
class Discriminator(nn.Module):
    
    def __init__(self, ngpu):
        
        super(Discriminator, self).__init__()
        
        self.ngpu = ngpu
        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity

In [8]:
def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [9]:
def main_loop(dataloader, netD, netG, epochs):

    img_list = []
    batches_done = 0
    fixed_noise = Variable(Tensor(np.random.normal(0, 1, (img_shape[0], 100))))
    iters = 0

    # Optimizers
    optimizer_G = torch.optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_D = torch.optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))

    Tensor = torch.cuda.FloatTensor if device.type == 'cuda' else torch.FloatTensor

    for epoch in range(epochs):
    
        for i, (imgs, _) in enumerate(dataloader):

            # Configure input
            real_imgs = Variable(imgs.type(Tensor))

            # ---------------------
            #  Train Discriminator
            # ---------------------

            optimizer_D.zero_grad()

            # Sample noise as generator input
            z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], 100))))

            # Generate a batch of images
            fake_imgs = netG(z)

            # Real images
            real_validity = netD(real_imgs)
            # Fake images
            fake_validity = netD(fake_imgs)
            # Gradient penalty
            gradient_penalty = compute_gradient_penalty(netD, real_imgs.data, fake_imgs.data)
            # Adversarial loss
            d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + 10 * gradient_penalty

            d_loss.backward()
            optimizer_D.step()

            optimizer_G.zero_grad()

            # Train the generator every n_critic steps
            if i % 5 == 0:

                # -----------------
                #  Train Generator
                # -----------------

                # Generate a batch of images
                fake_imgs = netG(z)
                # Loss measures generator's ability to fool the discriminator
                # Train on fake images
                fake_validity = netD(fake_imgs)
                g_loss = -torch.mean(fake_validity)

                g_loss.backward()
                optimizer_G.step()

                print("[Epoch %d/%d] [Loss_D: %.4f] [Loss_G: %.4f]" % (epoch, epochs, d_loss.item(), g_loss.item()))

                batches_done += 5
            
                # Check how the generator is doing by saving G's output on fixed_noise
                if (iters % 500 == 0) or ((epoch == epochs - 1) and (i == len(dataloader) - 1)):
                    
                    with torch.no_grad():

                        fake = netG(fixed_noise).detach().cpu()
                
                img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
                
                iters += 1

[Epoch 0/2000] [Batch 0/36] [D loss: 8.140008] [G loss: 0.076349]
[Epoch 0/2000] [Batch 5/36] [D loss: -6.494478] [G loss: 0.092500]
[Epoch 0/2000] [Batch 10/36] [D loss: -33.581726] [G loss: -0.277686]
[Epoch 0/2000] [Batch 15/36] [D loss: -60.010551] [G loss: -1.126941]
[Epoch 0/2000] [Batch 20/36] [D loss: -54.456230] [G loss: -2.051829]
[Epoch 0/2000] [Batch 25/36] [D loss: -62.852905] [G loss: -3.165140]
[Epoch 0/2000] [Batch 30/36] [D loss: -58.370480] [G loss: -4.270798]
[Epoch 0/2000] [Batch 35/36] [D loss: -68.422417] [G loss: -5.309797]
[Epoch 1/2000] [Batch 0/36] [D loss: -61.820526] [G loss: -7.062860]
[Epoch 1/2000] [Batch 5/36] [D loss: -52.657806] [G loss: -8.278675]
[Epoch 1/2000] [Batch 10/36] [D loss: -68.601868] [G loss: -8.708233]
[Epoch 1/2000] [Batch 15/36] [D loss: -73.149986] [G loss: -10.025081]
[Epoch 1/2000] [Batch 20/36] [D loss: -78.465897] [G loss: -12.286262]
[Epoch 1/2000] [Batch 25/36] [D loss: -77.040527] [G loss: -11.729839]
[Epoch 1/2000] [Batch 30/3

[Epoch 14/2000] [Batch 30/36] [D loss: -16.049292] [G loss: 6.798040]
[Epoch 14/2000] [Batch 35/36] [D loss: -15.606998] [G loss: 1.281393]
[Epoch 15/2000] [Batch 0/36] [D loss: -18.482231] [G loss: 1.766213]
[Epoch 15/2000] [Batch 5/36] [D loss: -18.632755] [G loss: -0.086165]
[Epoch 15/2000] [Batch 10/36] [D loss: -17.497238] [G loss: 3.012962]
[Epoch 15/2000] [Batch 15/36] [D loss: -19.321211] [G loss: 4.094294]
[Epoch 15/2000] [Batch 20/36] [D loss: -14.045141] [G loss: -2.167813]
[Epoch 15/2000] [Batch 25/36] [D loss: -20.105804] [G loss: 10.875488]
[Epoch 15/2000] [Batch 30/36] [D loss: -17.760672] [G loss: 2.638322]
[Epoch 15/2000] [Batch 35/36] [D loss: -18.550064] [G loss: 6.327609]
[Epoch 16/2000] [Batch 0/36] [D loss: -17.294380] [G loss: 2.440010]
[Epoch 16/2000] [Batch 5/36] [D loss: -16.614441] [G loss: -11.128002]
[Epoch 16/2000] [Batch 10/36] [D loss: -14.423401] [G loss: -3.809296]
[Epoch 16/2000] [Batch 15/36] [D loss: -15.825748] [G loss: -6.440878]
[Epoch 16/2000] [

[Epoch 29/2000] [Batch 15/36] [D loss: -8.828366] [G loss: 4.193177]
[Epoch 29/2000] [Batch 20/36] [D loss: -14.215649] [G loss: -2.086264]
[Epoch 29/2000] [Batch 25/36] [D loss: -12.250093] [G loss: -1.235298]
[Epoch 29/2000] [Batch 30/36] [D loss: -12.108724] [G loss: 4.689228]
[Epoch 29/2000] [Batch 35/36] [D loss: -11.110906] [G loss: -0.592358]
[Epoch 30/2000] [Batch 0/36] [D loss: -13.059868] [G loss: -4.701195]
[Epoch 30/2000] [Batch 5/36] [D loss: -10.025763] [G loss: 2.645938]
[Epoch 30/2000] [Batch 10/36] [D loss: -14.683952] [G loss: 8.008881]
[Epoch 30/2000] [Batch 15/36] [D loss: -12.593683] [G loss: 8.931295]
[Epoch 30/2000] [Batch 20/36] [D loss: -12.285444] [G loss: 15.661448]
[Epoch 30/2000] [Batch 25/36] [D loss: -13.407034] [G loss: 13.881634]
[Epoch 30/2000] [Batch 30/36] [D loss: -12.194681] [G loss: 10.679959]
[Epoch 30/2000] [Batch 35/36] [D loss: -14.238192] [G loss: 10.363945]
[Epoch 31/2000] [Batch 0/36] [D loss: -15.258483] [G loss: 12.340699]
[Epoch 31/2000]

[Epoch 44/2000] [Batch 0/36] [D loss: -12.273310] [G loss: -3.575813]
[Epoch 44/2000] [Batch 5/36] [D loss: -11.989341] [G loss: 3.931321]
[Epoch 44/2000] [Batch 10/36] [D loss: -11.839395] [G loss: -1.625475]
[Epoch 44/2000] [Batch 15/36] [D loss: -12.791260] [G loss: 9.995088]
[Epoch 44/2000] [Batch 20/36] [D loss: -12.880981] [G loss: 8.990170]
[Epoch 44/2000] [Batch 25/36] [D loss: -11.319191] [G loss: 1.336730]
[Epoch 44/2000] [Batch 30/36] [D loss: -14.328083] [G loss: 4.096736]
[Epoch 44/2000] [Batch 35/36] [D loss: -14.901880] [G loss: 8.990905]
[Epoch 45/2000] [Batch 0/36] [D loss: -9.337311] [G loss: 4.425880]
[Epoch 45/2000] [Batch 5/36] [D loss: -15.472692] [G loss: 7.176835]
[Epoch 45/2000] [Batch 10/36] [D loss: -13.637619] [G loss: 3.987604]
[Epoch 45/2000] [Batch 15/36] [D loss: -12.150116] [G loss: 8.872889]
[Epoch 45/2000] [Batch 20/36] [D loss: -11.974657] [G loss: -0.856950]
[Epoch 45/2000] [Batch 25/36] [D loss: -14.829085] [G loss: -4.263987]
[Epoch 45/2000] [Batc

[Epoch 58/2000] [Batch 25/36] [D loss: -11.187338] [G loss: 2.841147]
[Epoch 58/2000] [Batch 30/36] [D loss: -11.635683] [G loss: 3.793353]
[Epoch 58/2000] [Batch 35/36] [D loss: -12.305464] [G loss: 4.161177]
[Epoch 59/2000] [Batch 0/36] [D loss: -11.806597] [G loss: 2.863225]
[Epoch 59/2000] [Batch 5/36] [D loss: -11.974075] [G loss: 6.445105]
[Epoch 59/2000] [Batch 10/36] [D loss: -10.708133] [G loss: 8.827099]
[Epoch 59/2000] [Batch 15/36] [D loss: -12.450662] [G loss: 8.260624]
[Epoch 59/2000] [Batch 20/36] [D loss: -7.919599] [G loss: 9.609848]
[Epoch 59/2000] [Batch 25/36] [D loss: -12.292027] [G loss: 17.396404]
[Epoch 59/2000] [Batch 30/36] [D loss: -11.444623] [G loss: 1.942168]
[Epoch 59/2000] [Batch 35/36] [D loss: -11.814080] [G loss: -0.983091]
[Epoch 60/2000] [Batch 0/36] [D loss: -11.091570] [G loss: -6.200070]
[Epoch 60/2000] [Batch 5/36] [D loss: -10.412855] [G loss: -5.009209]
[Epoch 60/2000] [Batch 10/36] [D loss: -13.647948] [G loss: 4.961457]
[Epoch 60/2000] [Batc

...

[Epoch 1942/2000] [Batch 30/36] [D loss: -10.379781] [G loss: 1.217764]
[Epoch 1942/2000] [Batch 35/36] [D loss: -16.082781] [G loss: -0.201285]
[Epoch 1943/2000] [Batch 0/36] [D loss: -15.750201] [G loss: -0.310726]
[Epoch 1943/2000] [Batch 5/36] [D loss: -11.998199] [G loss: -0.101987]
[Epoch 1943/2000] [Batch 10/36] [D loss: -10.209638] [G loss: -3.290493]
[Epoch 1943/2000] [Batch 15/36] [D loss: -12.989634] [G loss: -3.499654]
[Epoch 1943/2000] [Batch 20/36] [D loss: -12.712061] [G loss: -1.931267]
[Epoch 1943/2000] [Batch 25/36] [D loss: -12.250648] [G loss: -3.385370]
[Epoch 1943/2000] [Batch 30/36] [D loss: -11.161728] [G loss: 3.439613]
[Epoch 1943/2000] [Batch 35/36] [D loss: -13.668476] [G loss: -1.241038]
[Epoch 1944/2000] [Batch 0/36] [D loss: -13.051662] [G loss: -0.921450]
[Epoch 1944/2000] [Batch 5/36] [D loss: -12.514750] [G loss: 1.951464]
[Epoch 1944/2000] [Batch 10/36] [D loss: -10.591139] [G loss: 0.048405]
[Epoch 1944/2000] [Batch 15/36] [D loss: -14.655071] [G los

[Epoch 1957/2000] [Batch 0/36] [D loss: -12.936407] [G loss: -1.761411]
[Epoch 1957/2000] [Batch 5/36] [D loss: -13.260396] [G loss: 2.655050]
[Epoch 1957/2000] [Batch 10/36] [D loss: -13.185867] [G loss: 0.428086]
[Epoch 1957/2000] [Batch 15/36] [D loss: -12.541411] [G loss: -0.188766]
[Epoch 1957/2000] [Batch 20/36] [D loss: -13.912264] [G loss: -1.428766]
[Epoch 1957/2000] [Batch 25/36] [D loss: -11.679541] [G loss: -3.854665]
[Epoch 1957/2000] [Batch 30/36] [D loss: -12.993379] [G loss: -1.671434]
[Epoch 1957/2000] [Batch 35/36] [D loss: -15.290096] [G loss: 1.230591]
[Epoch 1958/2000] [Batch 0/36] [D loss: -13.053588] [G loss: 1.914274]
[Epoch 1958/2000] [Batch 5/36] [D loss: -12.143719] [G loss: -3.458850]
[Epoch 1958/2000] [Batch 10/36] [D loss: -13.802992] [G loss: -2.703309]
[Epoch 1958/2000] [Batch 15/36] [D loss: -12.330232] [G loss: -3.766716]
[Epoch 1958/2000] [Batch 20/36] [D loss: -12.608734] [G loss: 0.337275]
[Epoch 1958/2000] [Batch 25/36] [D loss: -13.064628] [G loss

[Epoch 1971/2000] [Batch 10/36] [D loss: -13.348152] [G loss: -0.131691]
[Epoch 1971/2000] [Batch 15/36] [D loss: -13.200214] [G loss: -1.346019]
[Epoch 1971/2000] [Batch 20/36] [D loss: -8.987201] [G loss: -1.117267]
[Epoch 1971/2000] [Batch 25/36] [D loss: -12.158667] [G loss: -2.851085]
[Epoch 1971/2000] [Batch 30/36] [D loss: -11.635969] [G loss: -1.707492]
[Epoch 1971/2000] [Batch 35/36] [D loss: -13.414579] [G loss: -6.367953]
[Epoch 1972/2000] [Batch 0/36] [D loss: -12.896144] [G loss: -6.234357]
[Epoch 1972/2000] [Batch 5/36] [D loss: -14.556744] [G loss: -1.081415]
[Epoch 1972/2000] [Batch 10/36] [D loss: -12.177932] [G loss: -2.092560]
[Epoch 1972/2000] [Batch 15/36] [D loss: -11.402531] [G loss: 0.188093]
[Epoch 1972/2000] [Batch 20/36] [D loss: -13.433288] [G loss: -1.611131]
[Epoch 1972/2000] [Batch 25/36] [D loss: -14.309793] [G loss: 0.399370]
[Epoch 1972/2000] [Batch 30/36] [D loss: -11.614765] [G loss: -1.790717]
[Epoch 1972/2000] [Batch 35/36] [D loss: -13.266237] [G 

[Epoch 1985/2000] [Batch 20/36] [D loss: -11.972765] [G loss: 3.450821]
[Epoch 1985/2000] [Batch 25/36] [D loss: -12.929920] [G loss: -2.615705]
[Epoch 1985/2000] [Batch 30/36] [D loss: -11.802591] [G loss: -1.203551]
[Epoch 1985/2000] [Batch 35/36] [D loss: -12.257883] [G loss: -1.081411]
[Epoch 1986/2000] [Batch 0/36] [D loss: -11.815852] [G loss: -3.400924]
[Epoch 1986/2000] [Batch 5/36] [D loss: -14.089633] [G loss: 1.246867]
[Epoch 1986/2000] [Batch 10/36] [D loss: -10.107473] [G loss: -3.213270]
[Epoch 1986/2000] [Batch 15/36] [D loss: -12.951866] [G loss: 0.513760]
[Epoch 1986/2000] [Batch 20/36] [D loss: -12.139552] [G loss: -0.216120]
[Epoch 1986/2000] [Batch 25/36] [D loss: -12.922075] [G loss: -4.201471]
[Epoch 1986/2000] [Batch 30/36] [D loss: -12.249763] [G loss: -5.044462]
[Epoch 1986/2000] [Batch 35/36] [D loss: -13.977646] [G loss: -1.626818]
[Epoch 1987/2000] [Batch 0/36] [D loss: -10.448598] [G loss: -1.382514]
[Epoch 1987/2000] [Batch 5/36] [D loss: -14.175322] [G lo

[Epoch 1999/2000] [Batch 30/36] [D loss: -10.230433] [G loss: -4.491202]
[Epoch 1999/2000] [Batch 35/36] [D loss: -13.971437] [G loss: -2.766938]


In [ ]:
def show_results(dataloader, img_list):
    
    # Grab a batch of real images from the dataloader
    real_batch = next(iter(dataloader))

    # Plot the real images
    plt.figure(figsize=(15,15))
    plt.subplot(1,2,1)
    plt.axis("off")
    plt.title("Real Images")
    plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

    # Plot the fake images from the last epoch
    plt.subplot(1,2,2)
    plt.axis("off")
    plt.title("Fake Images")
    plt.imshow(np.transpose(img_list[-1],(1,2,0)))
    plt.show()
    

In [ ]:
def do_run(dataloader, epochs):

    netG = Generator(ngpu).to(device)
    netD = Discriminator(ngpu).to(device)

    img_list = main_loop(dataloader, netG, netD, epochs)
    show_results(dataloader, img_list)

In [ ]:
do_run(dataloader_mountains, 50)
